### Appendix B: Mixed Integer Programming

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from IPython.display import display

In [2]:
np.random.seed(0)
n = 5
A = np.zeros([n,n])
for j in range(5):
    #do a multiplier times 2 for staying in the center of the city
    A[0] = [np.random.random()*(1+int(x==0)) for x in range(5)]
    A[0] = A[0]/np.sum(A[0])

for i in range(1,5):
    #if the location is close then do a multiplier
    A[i] =[np.random.random()*3]+[np.random.random()*(3-int((abs(i-j)+1)/2)) for j in range(1,5)]
    A[i] = A[i]/np.sum(A[i])

In [3]:
display(np.round(A,3))

array([[0.475, 0.194, 0.112, 0.19 , 0.029],
       [0.337, 0.075, 0.332, 0.183, 0.073],
       [0.162, 0.317, 0.28 , 0.233, 0.008],
       [0.218, 0.144, 0.145, 0.333, 0.16 ],
       [0.214, 0.087, 0.277, 0.024, 0.398]])

In [4]:
np.random.seed(0)
#arbitraty optimal ditribution
O = np.array([42,17,13,20,8])*5
#random vector to randomly create initial distribution
d = np.round(np.random.normal(0, 15, 5))
#initial distribution
x0 = np.round((O+d)/np.sum(O+d)*500)
#B matrix described in the assignment
B = np.zeros([n**2,n])
for i in range(n**2):
    for j in range(n):
        a = int(i/n)
        b = (i)%n
        if j == a:
            if b != a:
                B[i][j] = -1
        if j == b:
            if b != a:
                B[i][j] = 1

In [5]:
#K matrix to ensure that we don't relocate more scooters than
#what there are in a given location
K = np.zeros([n,n**2])
for i in range(n):
    a = np.zeros(n**2)
    for z in range(n):
        a[i*n+z]=1
    K[i,:] = a
K = K.transpose()

In [6]:
#multiplier
s = 5
#coefficient of the cost of relocating the scooter
C = np.ones(n**2)*s
#input varibale
u = cp.Variable(n**2, integer=True)
#objective and constraints as defined in paper
objective = cp.Minimize(cp.sum_squares(O-A@x0-u@B)+C@u)
constraints = [
               np.ones(n**2)@u <= 100,
                u >= 0,
                u@K-x0 <= 0
               ]
prob = cp.Problem(objective,
                 constraints)
solution = prob.solve()
sol = np.round(u.value)
print('C =  I*{0} . Solution = {1}'.format(s,np.round(solution,2)))
display(pd.DataFrame(sol.reshape(5,5)))

C =  I*5 . Solution = 989.54


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.0
1,20.0,0.0,0.0,0.0,0.0
2,23.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,39.0,0.0,0.0,1.0,0.0
